In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [ ]:
DATA = '/content/drive/MyDrive/dataHN/data'
METADATA = '/content/drive/MyDrive/dataHN/metadata'
EXPERIMENTS = '/content/drive/MyDrive/dataHN/experiments'

In [ ]:
def crop_and_save(image_path, bbox, output_path):
    image = Image.open(image_path)

    x_min, y_min, x_max, y_max = bbox
    cropped = image.crop((x_min, y_min, x_max, y_max))
    cropped.save(output_path)

# Corpus docs

In [ ]:
dir_img = "/content/drive/MyDrive/dataHN/data/cleaned/doc_imgs_cleaned"



## Grounding-DINO ill crops

In [ ]:

det_csv_path = "/content/drive/MyDrive/dataHN/experiments/dino_ill_detect/results_tiny/detections.csv"
crops_dir = os.path.join(DATA, 'processed', 'g-dino_doc_crops')
os.makedirs(crops_dir, exist_ok=True)

In [ ]:
df = pd.read_csv(det_csv_path)

In [ ]:
df.columns

Index(['image_file', 'label', 'score', 'x_min', 'y_min', 'x_max', 'y_max'], dtype='object')

In [ ]:
df_detections = df.copy()

df_detections["bbox_id"] = df_detections.groupby("image_file").cumcount() + 1

df_detections["detection_id"] = df_detections.apply(
    lambda row: f"{os.path.splitext(row['image_file'])[0]}_{row['bbox_id']:02d}", axis=1)

df_detections["bbox_list"] = df_detections[["x_min","y_min","x_max","y_max"]].values.tolist()

In [ ]:
df_detections.head(2)

,image_file,label,score,x_min,y_min,x_max,y_max,bbox_id,detection_id,bbox_list
0,Maciet_142-1_115.jpg,illustration,0.576,872.61,561.42,1755.11,1325.31,1,Maciet_142-1_115_01,"[872.61, 561.42, 1755.11, 1325.31]"
1,Maciet_142-1_115.jpg,illustration,0.412,648.79,223.04,4214.58,5539.15,2,Maciet_142-1_115_02,"[648.79, 223.04, 4214.58, 5539.15]"


In [ ]:
# Calculer crops
os.makedirs(crops_dir, exist_ok=True)
for idx, row in tqdm(df_detections.iterrows()):
    image_path = os.path.join(dir_img, row['image_file'])
    detection_path = os.path.join(crops_dir, f"{row['detection_id']}.jpg")
    bbox_list = row['bbox_list']

    crop_and_save(image_path, bbox_list, detection_path)


2985it [20:12,  1.55s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131542390 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
2995it [20:26,  1.27s/it]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131396860 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
4596it [35:09,  2.18it/s]


In [ ]:
path_det_csv = os.path.join(METADATA, 'doc_detections.csv')
df_detections.to_csv(path_det_csv, index=False)

## Yolo ill crops

In [ ]:
yolo_csv = os.path.join(EXPERIMENTS, 'yolo_doc_ill_detect/predicts/csv/run8_yolo11m_prediction_results.csv')

In [ ]:
df_yolo_ill = pd.read_csv(yolo_csv, encoding='UTF-8')

In [ ]:
df_yolo_ill.head(5)

,image_file,class_id,class_name,score,x1,y1,x2,y2,bbox_index
0,Beaumont-Collinot_1859_145.jpg,0,illustration,0.904017,549.692993,618.003601,2219.839355,3050.201172,0
1,Bourgoin_1873A_103.jpg,0,illustration,0.930724,405.313965,1527.189209,1575.591309,2064.643555,0
2,Bourgoin_1873A_103.jpg,0,illustration,0.848927,415.865356,316.898651,1583.210938,1481.160645,1
3,Soldi_1881_271.jpg,0,illustration,0.837377,0.000000,0.000000,1830.151611,2922.183350,0
4,Maciet_142-1_049.jpg,0,illustration,0.835487,1666.919434,324.244873,2887.704590,1653.889526,0


In [ ]:

df_yolo_ill["bbox_id"] = df_yolo_ill.groupby("image_file").cumcount() + 1

df_yolo_ill["detection_id"] = df_yolo_ill.apply(
    lambda row: f"{os.path.splitext(row['image_file'])[0]}_{row['bbox_id']:02d}", axis=1)

df_yolo_ill["bbox_list"] = df_yolo_ill[["x1","y1","x2","y2"]].values.tolist()

In [ ]:
df_yolo_ill.head(2)

,image_file,class_id,class_name,score,x1,y1,x2,y2,bbox_index,bbox_id,detection_id,bbox_list
0,Beaumont-Collinot_1859_145.jpg,0,illustration,0.904017,549.692993,618.003601,2219.839355,3050.201172,0,1,Beaumont-Collinot_1859_145_01,"[549.6929931640625, 618.0036010742188, 2219.83..."
1,Bourgoin_1873A_103.jpg,0,illustration,0.930724,405.313965,1527.189209,1575.591309,2064.643555,0,1,Bourgoin_1873A_103_01,"[405.31396484375, 1527.189208984375, 1575.5913..."


In [ ]:
# Calculer crops
yolo_crops_dir = os.path.join(DATA, 'processed', 'yolo_doc_crops')

os.makedirs(yolo_crops_dir, exist_ok=True)
for idx, row in tqdm(df_yolo_ill.iterrows()):
    image_path = os.path.join(dir_img, row['image_file'])
    detection_path = os.path.join(yolo_crops_dir, f"{row['detection_id']}.jpg")
    bbox_list = row['bbox_list']

    crop_and_save(image_path, bbox_list, detection_path)

1729it [14:20,  3.53it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131396860 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
5757it [44:27,  2.53it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131542390 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
7705it [59:14,  2.17it/s]


In [ ]:
path_det_yolo_csv = os.path.join(METADATA, 'yolo_ill_crops.csv')
df_yolo_ill.to_csv(path_det_yolo_csv, index=False)

# Corpus objets

In [ ]:
DATA = '/content/drive/MyDrive/dataHN/data'
METADATA = '/content/drive/MyDrive/dataHN/metadata'
det_csv_path = "/content/drive/MyDrive/dataHN/experiments/dino_obj_detect/results_tiny/detections.csv"
dir_img = os.path.join(DATA, 'cleaned','obj_imgs_cleaned')
crops_dir = os.path.join(DATA, 'processed', 'g-dino_obj_crops')
os.makedirs(crops_dir, exist_ok=True)

In [ ]:
df = pd.read_csv(det_csv_path)

In [ ]:
df.columns

Index(['image_file', 'label', 'score', 'x_min', 'y_min', 'x_max', 'y_max'], dtype='object')

In [ ]:
df_detections = df.copy()

df_detections["bbox_id"] = df_detections.groupby("image_file").cumcount() + 1

df_detections["detection_id"] = df_detections.apply(
    lambda row: f"{os.path.splitext(row['image_file'])[0]}_{row['bbox_id']:02d}", axis=1)

df_detections["bbox_list"] = df_detections[["x_min","y_min","x_max","y_max"]].values.tolist()

In [ ]:
df_detections.head(2)

,image_file,label,score,x_min,y_min,x_max,y_max,bbox_id,detection_id,bbox_list
0,Christies_2005-04-21_697_01.jpg,object,0.808,32.31,30.89,489.67,486.41,1,Christies_2005-04-21_697_01_01,"[32.31, 30.89, 489.67, 486.41]"
1,GazetteDrouot_15085424_02.jpg,object,0.847,188.36,255.27,1526.08,1580.57,1,GazetteDrouot_15085424_02_01,"[188.36, 255.27, 1526.08, 1580.57]"


In [ ]:
# Calculer crops
os.makedirs(crops_dir, exist_ok=True)
for idx, row in tqdm(df_detections.iterrows()):
    image_path = os.path.join(dir_img, row['image_file'])
    detection_path = os.path.join(crops_dir, f"{row['detection_id']}.jpg")
    bbox_list = row['bbox_list']

    crop_and_save(image_path, bbox_list, detection_path)


2127it [09:53,  3.58it/s]


In [ ]:
path_det_csv = os.path.join(METADATA, 'obj_detections.csv')
df_detections.to_csv(path_det_csv, index=False)